<a href="https://colab.research.google.com/github/jagadeeshD3/Spark-End-to-end-analysis/blob/main/pyspark_end_to_end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=d309828e61b1297b1163315495271f77ab22e45e4010eae2ce0021df1b61da1c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("spark end to end project").getOrCreate()

print(spark)

In [ ]:
print(spark.version)

3.5.1


In [ ]:
import pandas as pd
import numpy as np

#create pd_temp
pd_temp = pd.DataFrame(np.random.random(10))

#create spark_temp from pd_temp
spark_temp = spark.createDataFrame(pd_temp)

#Examine the tables in the catalog
print(spark.catalog.listTables())

#Add spark_temp to the catalog
spark_temp.createOrReplaceTempView("temp")

#Examine the tables in the spark catalog
print(spark.catalog.listTables())

[]
[Table(name='temp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]


In [ ]:
#Read csv file
airports = spark.read.csv("/content/airports.csv",header=True,inferSchema=True)

In [ ]:
airports.show(5)

+---+--------------------+----------+-----------+----+---+---+
|faa|                name|       lat|        lon| alt| tz|dst|
+---+--------------------+----------+-----------+----+---+---+
|04G|   Lansdowne Airport|41.1304722|-80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|41.9893408|-88.1012428| 801| -6|  A|
|06N|     Randall Airport| 41.431912|-74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|31.0744722|-81.4277778|  11| -4|  A|
+---+--------------------+----------+-----------+----+---+---+
only showing top 5 rows



In [ ]:
type(airports)

pyspark.sql.dataframe.DataFrame

In [ ]:
spark.catalog.listDatabases

<bound method Catalog.listDatabases of <pyspark.sql.catalog.Catalog object at 0x79b376736020>>

In [ ]:
flights = spark.read.csv("/content/flights_small.csv",header = True,inferSchema=True)

In [ ]:
flights.show(10)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

In [ ]:
flights.createOrReplaceTempView("flights")

In [ ]:
spark.catalog.listTables()

[Table(name='flights', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='temp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [ ]:
flights = flights.withColumn("duration_hrs",flights.air_time/60)

In [ ]:
flights.show(10)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|               2.2|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               6.0|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|              1.85|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|1.3833333333333333|
|2014|    3|  9|     754|  

In [ ]:
flights.describe().show()

+-------+------+------------------+-----------------+------------------+------------------+------------------+------------------+-------+-------+-----------------+------+-----+------------------+-----------------+------------------+-----------------+------------------+
|summary|  year|             month|              day|          dep_time|         dep_delay|          arr_time|         arr_delay|carrier|tailnum|           flight|origin| dest|          air_time|         distance|              hour|           minute|      duration_hrs|
+-------+------+------------------+-----------------+------------------+------------------+------------------+------------------+-------+-------+-----------------+------+-----+------------------+-----------------+------------------+-----------------+------------------+
|  count| 10000|             10000|            10000|             10000|             10000|             10000|             10000|  10000|  10000|            10000| 10000|10000|             1

In [ ]:
#Filter flights with a SQL
long_flights1 = flights.filter('distance>1000')
long_flights1.show(10)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|     duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|              6.0|
|2014|    4| 19|    1236|       -4|    1508|       -7|     AS| N309AS|   490|   SEA| SAN|     135|    1050|  12|    36|             2.25|
|2014|   11| 19|    1812|       -3|    2352|       -4|     AS| N564AS|    26|   SEA| ORD|     198|    1721|  18|    12|              3.3|
|2014|    8|  3|    1120|        0|    1415|        2|     AS| N305AS|   656|   SEA| PHX|     154|    1107|  11|    20|2.566666666666667|
|2014|   11| 12|    2346|       -4

In [ ]:
#Filter flights with a boolean column
long_flights2 = flights.filter(flights.distance >1000)
long_flights2.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               6.0|
|2014|    4| 19|    1236|       -4|    1508|       -7|     AS| N309AS|   490|   SEA| SAN|     135|    1050|  12|    36|              2.25|
|2014|   11| 19|    1812|       -3|    2352|       -4|     AS| N564AS|    26|   SEA| ORD|     198|    1721|  18|    12|               3.3|
|2014|    8|  3|    1120|        0|    1415|        2|     AS| N305AS|   656|   SEA| PHX|     154|    1107|  11|    20| 2.566666666666667|
|2014|   11| 12|    2346|  

In [ ]:
#Select the columns as a string
selected1 = flights.select('tailnum','origin','dest')
selected1.show()

+-------+------+----+
|tailnum|origin|dest|
+-------+------+----+
| N846VA|   SEA| LAX|
| N559AS|   SEA| HNL|
| N847VA|   SEA| SFO|
| N360SW|   PDX| SJC|
| N612AS|   SEA| BUR|
| N646SW|   PDX| DEN|
| N422WN|   PDX| OAK|
| N361VA|   SEA| SFO|
| N309AS|   SEA| SAN|
| N564AS|   SEA| ORD|
| N323AS|   SEA| LAX|
| N305AS|   SEA| PHX|
| N433AS|   SEA| LAS|
| N765AS|   SEA| ANC|
| N713AS|   SEA| SFO|
| N27205|   PDX| SFO|
| N626AS|   SEA| SMF|
| N8634A|   SEA| MDW|
| N597AS|   SEA| BOS|
| N215AG|   PDX| BUR|
+-------+------+----+
only showing top 20 rows



In [ ]:
#Select the columns using df.col
selected2 = flights.select(flights.tailnum,flights.origin,flights.dest)
selected2.show()

+-------+------+----+
|tailnum|origin|dest|
+-------+------+----+
| N846VA|   SEA| LAX|
| N559AS|   SEA| HNL|
| N847VA|   SEA| SFO|
| N360SW|   PDX| SJC|
| N612AS|   SEA| BUR|
| N646SW|   PDX| DEN|
| N422WN|   PDX| OAK|
| N361VA|   SEA| SFO|
| N309AS|   SEA| SAN|
| N564AS|   SEA| ORD|
| N323AS|   SEA| LAX|
| N305AS|   SEA| PHX|
| N433AS|   SEA| LAS|
| N765AS|   SEA| ANC|
| N713AS|   SEA| SFO|
| N27205|   PDX| SFO|
| N626AS|   SEA| SMF|
| N8634A|   SEA| MDW|
| N597AS|   SEA| BOS|
| N215AG|   PDX| BUR|
+-------+------+----+
only showing top 20 rows



In [ ]:
selected3 = flights.filter((flights.origin=='SEA') & (flights.dest == 'PDX'))
selected3.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|       duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------------------+
|2014|   10|  1|    1717|       -8|    1819|        4|     OO| N810SK|  4546|   SEA| PDX|      28|     129|  17|    17| 0.4666666666666667|
|2014|    9| 26|    2339|      144|      29|      142|     OO| N822SK|  4612|   SEA| PDX|      29|     129|  23|    39|0.48333333333333334|
|2014|    8| 18|    1728|       -2|    1822|        0|     OO| N586SW|  5440|   SEA| PDX|      41|     129|  17|    28| 0.6833333333333333|
|2014|    2|  4|    2053|       -7|    2144|       -4|     OO| N223SW|  5433|   SEA| PDX|      29|     129|  20|    53|0.48333333333333334|
|2014|    2|  9|    

In [ ]:
#Create a table of the average speed of each flight both ways.
#Calculate average speed by dividing the distance by the air_time (converted to hours).Use the .alias() method name
# Define avg_speed
avg_speed = flights.withColumn("avg_speed",(flights.distance*60)/flights.air_time)
avg_speed.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|         avg_speed|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|               2.2| 433.6363636363636|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               6.0| 446.1666666666667|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|              1.85|367.02702702702703|
|2014|    4|  9|    1705|       45|    1839|       3

In [ ]:
avg_speed.show(10)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|         avg_speed|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|               2.2| 433.6363636363636|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               6.0| 446.1666666666667|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|              1.85|367.02702702702703|
|2014|    4|  9|    1705|       45|    1839|       3

In [ ]:
from pyspark.sql.functions import col, expr
avg_speed = flights.select("origin","dest","tailnum",expr("distance*60/air_time").alias("avg_speed"))
avg_speed.show()

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
|   PDX| SJC| N360SW|411.32530120481925|
|   SEA| BUR| N612AS| 442.6771653543307|
|   PDX| DEN| N646SW|491.40495867768595|
|   PDX| OAK| N422WN|             362.0|
|   SEA| SFO| N361VA| 415.7142857142857|
|   SEA| SAN| N309AS| 466.6666666666667|
|   SEA| ORD| N564AS| 521.5151515151515|
|   SEA| LAX| N323AS| 440.3076923076923|
|   SEA| PHX| N305AS| 431.2987012987013|
|   SEA| LAS| N433AS| 409.6062992125984|
|   SEA| ANC| N765AS|474.75409836065575|
|   SEA| SFO| N713AS| 315.8139534883721|
|   PDX| SFO| N27205| 366.6666666666667|
|   SEA| SMF| N626AS|477.63157894736844|
|   SEA| MDW| N8634A| 481.3888888888889|
|   SEA| BOS| N597AS| 516.4137931034483|
|   PDX| BUR| N215AG| 441.6216216216216|
+------+----+-------+------------------+
only showing top

In [ ]:
#Approach 2
avg_speed = flights.select(flights.origin, flights.dest, flights.tailnum, (flights.distance*60/flights.air_time).alias("avg_speed"))

In [ ]:
avg_speed.show()

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
|   PDX| SJC| N360SW|411.32530120481925|
|   SEA| BUR| N612AS| 442.6771653543307|
|   PDX| DEN| N646SW|491.40495867768595|
|   PDX| OAK| N422WN|             362.0|
|   SEA| SFO| N361VA| 415.7142857142857|
|   SEA| SAN| N309AS| 466.6666666666667|
|   SEA| ORD| N564AS| 521.5151515151515|
|   SEA| LAX| N323AS| 440.3076923076923|
|   SEA| PHX| N305AS| 431.2987012987013|
|   SEA| LAS| N433AS| 409.6062992125984|
|   SEA| ANC| N765AS|474.75409836065575|
|   SEA| SFO| N713AS| 315.8139534883721|
|   PDX| SFO| N27205| 366.6666666666667|
|   SEA| SMF| N626AS|477.63157894736844|
|   SEA| MDW| N8634A| 481.3888888888889|
|   SEA| BOS| N597AS| 516.4137931034483|
|   PDX| BUR| N215AG| 441.6216216216216|
+------+----+-------+------------------+
only showing top

In [ ]:
speed_2 =flights.selectExpr('origin','dest','tailnum','distance/(air_time/60) as avg_speed')
speed_2.show()

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
|   PDX| SJC| N360SW| 411.3253012048193|
|   SEA| BUR| N612AS| 442.6771653543307|
|   PDX| DEN| N646SW|491.40495867768595|
|   PDX| OAK| N422WN|             362.0|
|   SEA| SFO| N361VA| 415.7142857142857|
|   SEA| SAN| N309AS| 466.6666666666667|
|   SEA| ORD| N564AS| 521.5151515151515|
|   SEA| LAX| N323AS| 440.3076923076923|
|   SEA| PHX| N305AS|431.29870129870125|
|   SEA| LAS| N433AS| 409.6062992125984|
|   SEA| ANC| N765AS|474.75409836065575|
|   SEA| SFO| N713AS| 315.8139534883721|
|   PDX| SFO| N27205| 366.6666666666667|
|   SEA| SMF| N626AS|477.63157894736844|
|   SEA| MDW| N8634A|481.38888888888886|
|   SEA| BOS| N597AS| 516.4137931034483|
|   PDX| BUR| N215AG| 441.6216216216216|
+------+----+-------+------------------+
only showing top

In [ ]:
#arr_time: string and distance: string, so to find min() and max() we need to convert this float
flights = flights.withColumn("distance",flights.distance.cast('float'))
flights = flights.withColumn("air_time",flights.air_time.cast('float'))

flights.describe('distance','air_time').show()

+-------+-----------------+------------------+
|summary|         distance|          air_time|
+-------+-----------------+------------------+
|  count|            10000|              9925|
|   mean|        1208.1516|152.88423173803525|
| stddev|656.8599023464376|  72.8656286392139|
|    min|             93.0|              20.0|
|    max|           2724.0|             409.0|
+-------+-----------------+------------------+



In [ ]:
#Find the length of the shortest (in terms of distance) flight that left PDX
flights.filter(flights.origin=='PDX').groupBy("origin").min("distance").show()

+------+-------------+
|origin|min(distance)|
+------+-------------+
|   PDX|        106.0|
+------+-------------+



In [ ]:
#Find the length of the longest (in terms of time) flight that left SEA
flights.filter(flights.origin=='SEA').groupBy().max("air_time").show()

+-------------+
|max(air_time)|
+-------------+
|        409.0|
+-------------+



In [ ]:
#get the average air time of Delta Airlines flights  that left SEA.
flights.filter((flights.carrier == "DL") & (flights.origin == 'SEA')).groupBy().avg("air_time").show()

+------------------+
|     avg(air_time)|
+------------------+
|188.20689655172413|
+------------------+



In [ ]:
#get the total number of hours all planes in this dataset spent in the air by creating a column called duration_hrs
flights.withColumn("duration_hrs",flights.air_time/60).groupBy().sum('duration_hrs').show()

+------------------+
| sum(duration_hrs)|
+------------------+
|25289.600000000126|
+------------------+



In [ ]:
#Group by tailnum column
by_plane = flights.groupBy("tailnum")


GroupedData[grouping expressions: [tailnum], value: [year: int, month: int ... 15 more fields], type: GroupBy]


In [ ]:
#Use the .count() method with no arguments to count the number of flights each plane made
by_plane.count().show()

+-------+-----+
|tailnum|count|
+-------+-----+
| N442AS|   38|
| N102UW|    2|
| N36472|    4|
| N38451|    4|
| N73283|    4|
| N513UA|    2|
| N954WN|    5|
| N388DA|    3|
| N567AA|    1|
| N516UA|    2|
| N927DN|    1|
| N8322X|    1|
| N466SW|    1|
|  N6700|    1|
| N607AS|   45|
| N622SW|    4|
| N584AS|   31|
| N914WN|    4|
| N654AW|    2|
| N336NW|    1|
+-------+-----+
only showing top 20 rows



In [ ]:
#group by origin column
by_origin = flights.groupBy("origin")
by_origin.count().show()

+------+-----+
|origin|count|
+------+-----+
|   SEA| 6754|
|   PDX| 3246|
+------+-----+



In [ ]:
#Find the .avg() of the air_time column to find average duration of flights from PDX and SEA
by_origin.avg('air_time').show()


+------+------------------+
|origin|     avg(air_time)|
+------+------------------+
|   SEA| 160.4361496051259|
|   PDX|137.11543248288737|
+------+------------------+



In [ ]:
# Import pyspark.sql.functions as F
import pyspark.sql.functions as F

#convert to dep_delay to numeric column
flights = flights.withColumn('dep_delay', flights.dep_delay.cast('float'))

# Group by month and dest
by_month_dest = flights.groupBy('month', 'dest')

In [ ]:
# Average departure delay by month and destination
by_month_dest.avg('dep_delay').show()

+-----+----+-------------------+
|month|dest|     avg(dep_delay)|
+-----+----+-------------------+
|    4| PHX| 1.6833333333333333|
|    1| RDM|             -1.625|
|    5| ONT| 3.5555555555555554|
|    7| OMA|               -6.5|
|    8| MDW|               7.45|
|    6| DEN|  5.418181818181818|
|    5| IAD|               -4.0|
|   12| COS|               -1.0|
|   11| ANC|  7.529411764705882|
|    5| AUS|              -0.75|
|    5| COS| 11.666666666666666|
|    2| PSP|                0.6|
|    4| ORD|0.14285714285714285|
|   10| DFW| 18.176470588235293|
|   10| DCA|               -1.5|
|    8| JNU|             18.125|
|   11| KOA|               -1.0|
|   10| OMA|-0.6666666666666666|
|    6| ONT|              9.625|
|    3| MSP|                3.2|
+-----+----+-------------------+
only showing top 20 rows



In [ ]:
airports.show()

+---+--------------------+----------------+-----------------+----+---+---+
|faa|                name|             lat|              lon| alt| tz|dst|
+---+--------------------+----------------+-----------------+----+---+---+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|
|0S9|Jefferson County ...|      48.0538086|     -122.8106436| 108| -8|  A|
|0W3|Harford County Ai...

In [ ]:
# Rename the faa column
airports = airports.withColumnRenamed('faa','dest')

In [ ]:
#Join the flights and airports dataframes
flights_with_airports = flights.join(airports,on="dest",how='leftouter')

In [ ]:
flights_with_airports.show()

+----+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+--------+--------+----+------+------------------+--------------------+---------+-----------+----+---+---+
|dest|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|air_time|distance|hour|minute|      duration_hrs|                name|      lat|        lon| alt| tz|dst|
+----+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+--------+--------+----+------+------------------+--------------------+---------+-----------+----+---+---+
| LAX|2014|   12|  8|     658|     -7.0|     935|       -5|     VX| N846VA|  1780|   SEA|   132.0|   954.0|   6|    58|               2.2|    Los Angeles Intl|33.942536|-118.408075| 126| -8|  A|
| HNL|2014|    1| 22|    1040|      5.0|    1505|        5|     AS| N559AS|   851|   SEA|   360.0|  2677.0|  10|    40|               6.0|       Honolulu Intl|21.318681|-157.922428|  13|-10|  N|
| SFO|2014|    3|  9|    

In [ ]:
planes = spark.read.csv("/content/planes.csv",header=True,inferSchema=True)

In [ ]:
planes.show(10)

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N110UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA

In [ ]:
# Rename year column on panes to avoid duplicate column name
planes = planes.withColumnRenamed('year','plane_year')

In [ ]:
#join the flights and plane table use key as tailnum column
model_data = flights_with_airports.join(planes,on='tailnum',how='leftouter')
model_data.show(10)

+-------+----+----+-----+---+--------+---------+--------+---------+-------+------+------+--------+--------+----+------+------------------+--------------------+---------+-----------+----+---+---+----------+--------------------+------------+--------+-------+-----+-----+---------+
|tailnum|dest|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|air_time|distance|hour|minute|      duration_hrs|                name|      lat|        lon| alt| tz|dst|plane_year|                type|manufacturer|   model|engines|seats|speed|   engine|
+-------+----+----+-----+---+--------+---------+--------+---------+-------+------+------+--------+--------+----+------+------------------+--------------------+---------+-----------+----+---+---+----------+--------------------+------------+--------+-------+-----+-----+---------+
| N846VA| LAX|2014|   12|  8|     658|     -7.0|     935|       -5|     VX|  1780|   SEA|   132.0|   954.0|   6|    58|               2.2|    Los Angeles Intl|33.9

In [ ]:
model_data.describe().show()

+-------+-------+-----+------+------------------+-----------------+------------------+------------------+------------------+------------------+-------+-----------------+------+------------------+-----------------+------------------+-----------------+------------------+--------------------+------------------+-------------------+-----------------+------------------+-----+-----------------+--------------------+--------------------+-------------+--------------------+------------------+-----------------+-------------+
|summary|tailnum| dest|  year|             month|              day|          dep_time|         dep_delay|          arr_time|         arr_delay|carrier|           flight|origin|          air_time|         distance|              hour|           minute|      duration_hrs|                name|               lat|                lon|              alt|                tz|  dst|       plane_year|                type|        manufacturer|        model|             engines|             s

In [ ]:
model_data = model_data.withColumn("arr_delay", model_data.arr_delay.cast("Integer"))
model_data = model_data.withColumn("air_time",model_data.air_time.cast("integer"))
model_data = model_data.withColumn("month",model_data.month.cast("integer"))
model_data = model_data.withColumn("plane_year",model_data.plane_year.cast("integer"))

In [ ]:
model_data.describe("arr_delay","air_time","month","plane_year").show()

+-------+------------------+------------------+------------------+-----------------+
|summary|         arr_delay|          air_time|             month|       plane_year|
+-------+------------------+------------------+------------------+-----------------+
|  count|              9925|              9925|             10000|             9354|
|   mean|2.2530982367758186|152.88423173803525|            6.6438|2001.594398118452|
| stddev|31.074918600451877|  72.8656286392139|3.3191600205962097|58.92921992728455|
|    min|               -58|                20|                 1|                0|
|    max|               900|               409|                12|             2014|
+-------+------------------+------------------+------------------+-----------------+



In [ ]:
# Create a new column
model_data = model_data.withColumn("plane_age",model_data.year-model_data.plane_year)

In [ ]:
model_data.show(10)

+-------+----+----+-----+---+--------+---------+--------+---------+-------+------+------+--------+--------+----+------+------------------+--------------------+---------+-----------+----+---+---+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+
|tailnum|dest|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|air_time|distance|hour|minute|      duration_hrs|                name|      lat|        lon| alt| tz|dst|plane_year|                type|manufacturer|   model|engines|seats|speed|   engine|plane_age|
+-------+----+----+-----+---+--------+---------+--------+---------+-------+------+------+--------+--------+----+------+------------------+--------------------+---------+-----------+----+---+---+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+
| N846VA| LAX|2014|   12|  8|     658|     -7.0|     935|       -5|     VX|  1780|   SEA|     132|   954.0|   6|    58|              

In [ ]:
model_data = model_data.withColumn('is_late', model_data.arr_delay >0)

model_data = model_data.withColumn('label', model_data.is_late.cast('integer'))

model_data.filter("arr_delay is not NULL and dep_delay is not NULL and air_time is not NULL and plane_year is not NULL").show()

+-------+----+----+-----+---+--------+---------+--------+---------+-------+------+------+--------+--------+----+------+------------------+--------------------+---------+-----------+----+---+---+----------+--------------------+--------------+-----------+-------+-----+-----+---------+---------+-------+-----+
|tailnum|dest|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|air_time|distance|hour|minute|      duration_hrs|                name|      lat|        lon| alt| tz|dst|plane_year|                type|  manufacturer|      model|engines|seats|speed|   engine|plane_age|is_late|label|
+-------+----+----+-----+---+--------+---------+--------+---------+-------+------+------+--------+--------+----+------+------------------+--------------------+---------+-----------+----+---+---+----------+--------------------+--------------+-----------+-------+-----+-----+---------+---------+-------+-----+
| N846VA| LAX|2014|   12|  8|     658|     -7.0|     935|       -5|     VX| 

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [ ]:
#Create a StringIndexer
carr_indexer = StringIndexer(inputCol='carrier', outputCol='carrier_index')
#Create a OneHotEncoder
carr_encoder = OneHotEncoder(inputCol='carrier_index', outputCol='carr_fact')

In [ ]:
# encode the dest column just like you did above
dest_indexer = StringIndexer(inputCol='dest', outputCol='dest_index')
dest_encoder = OneHotEncoder(inputCol='dest_index', outputCol='dest_fact')

In [ ]:
#Assemble a Vector
from pyspark.ml.feature import VectorAssembler

In [ ]:
vec_assembler =VectorAssembler(inputCols=['month', 'air_time','carr_fact','dest_fact','plane_age'],
                              outputCol='features',handleInvalid="skip")

In [ ]:
# #### Create the pipeline
# You're finally ready to create a` Pipeline!` Pipeline is a class in the `pyspark.ml module` that combines all the Estimators and Transformers that you've already created.

from pyspark.ml import Pipeline

flights_pipe = Pipeline(stages=[dest_indexer, dest_encoder, carr_indexer, carr_encoder, vec_assembler])

In [ ]:
piped_data =flights_pipe.fit(model_data).transform(model_data)

In [ ]:
piped_data.show()

+-------+----+----+-----+---+--------+---------+--------+---------+-------+------+------+--------+--------+----+------+------------------+--------------------+---------+-----------+----+---+---+----------+--------------------+--------------+-----------+-------+-----+-----+---------+---------+-------+-----+----------+---------------+-------------+--------------+--------------------+
|tailnum|dest|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|air_time|distance|hour|minute|      duration_hrs|                name|      lat|        lon| alt| tz|dst|plane_year|                type|  manufacturer|      model|engines|seats|speed|   engine|plane_age|is_late|label|dest_index|      dest_fact|carrier_index|     carr_fact|            features|
+-------+----+----+-----+---+--------+---------+--------+---------+-------+------+------+--------+--------+----+------+------------------+--------------------+---------+-----------+----+---+---+----------+--------------------+----

In [ ]:
training, test = piped_data.randomSplit([.6, .4])

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()

In [ ]:
# #### Create the evaluator
# The first thing you need when doing cross validation for model selection is a way to compare different models. Luckily, the pyspark.ml.evaluation submodule has classes for evaluating different kinds of models. Your model is a binary classification model, so you'll be using the `BinaryClassificationEvaluator` from the `pyspark.ml.evaluation` module. This evaluator calculates the area under the ROC. This is a metric that combines the two kinds of errors a binary classifier can make (false positives and false negatives) into a simple number.

import pyspark.ml.evaluation as evals

evaluator = evals.BinaryClassificationEvaluator(metricName='areaUnderROC')

In [ ]:
# Import the tuning submodule
import pyspark.ml.tuning as tune

# Create the parameter grid
grid = tune.ParamGridBuilder()

# Add the hyperparameter
grid = grid.addGrid(lr.regParam, np.arange(0, .1, .01))
grid = grid.addGrid(lr.elasticNetParam, [0,1])

# Build the grid
grid = grid.build()

In [ ]:
# Create the CrossValidator
cv = tune.CrossValidator(estimator=lr,
               estimatorParamMaps=grid,
               evaluator=evaluator
               )

In [ ]:
# Fit cross validation models
models = cv.fit(training)

In [ ]:
# Extract the best model
best_lr = models.bestModel

In [ ]:
# Use the model to predict the test set
test_results = best_lr.transform(test)
#
# Evaluate the predictions
print(evaluator.evaluate(test_results))

0.6934787628845274
